In [50]:
import json
from pathlib import Path

import pandas as pd
import requests
import tomli

In [59]:
API_TOML_DIR = Path(Path.cwd().parent, "api_creds.toml")
WEATHER_LOCATION_MAPPING_DIR = Path(
    Path.cwd().parent, "data_information", "weather_location_mapping.json"
)
PTID_AREA_MAPPING_DIR = Path(
    Path.cwd().parent, "data_information", "PTID_name_mapping.json"
)

In [52]:
with open(API_TOML_DIR, "rb") as f:
    key = tomli.load(f)["api_key"]

In [53]:
def get_forecast(
    lat_lon: tuple[float, float], forecast_days: int = 3, api_key: str = key
) -> dict:
    """Gets a forecase for a given lat lon from the https://www.weatherapi.com/ site.
    Requires an api key to be defined.

    Args:
        lat_lon (tuple[float, float]): lat/lon of the location to be forecasted
        forecast_days (int, optional): number of days to forecast, note free tier weatherapi is restricted to 14 days. Defaults to 3.
        api_key (str, optional): api key for weather api. Defaults to key.

    Raises:
        SystemExit: generic error catch for incorrect request parameters

    Returns:
        dict: json of the returned api call#
    """
    BASE_URL = "https://api.weatherapi.com/v1/forecast.json?"
    str_lat_lon = ",".join([str(x) for x in lat_lon])
    query_params = {"q": str_lat_lon, "days": forecast_days, "key": api_key}
    try:
        response = requests.get(BASE_URL, params=query_params)
    except (
        requests.exceptions.RequestException
    ) as e:  # TODO Generic error catching = bad
        raise SystemExit(e)
    return response.json()

In [54]:
def parse_forcast_response(forecast_response_json: dict) -> dict:
    """From a full api response dict, extract the latitude, longitude, forecast dates and corresposnding
    min and max termperatures forecasted,

    Args:
        forecast_response_json (dict): full response from get_forecast function

    Returns:
        dict: latitude, longitude, dates, minmax temp dictionary
    """
    num_days_forecasted = len(forecast_response_json["forecast"]["forecastday"])
    lat = forecast_response_json["location"]["lat"]
    lon = forecast_response_json["location"]["lon"]
    dates = []
    max_temps = []
    min_temps = []
    for day in range(num_days_forecasted):
        dates.append(forecast_response_json["forecast"]["forecastday"][day]["date"])
        max_temps.append(
            forecast_response_json["forecast"]["forecastday"][day]["day"]["maxtemp_c"]
        )
        min_temps.append(
            forecast_response_json["forecast"]["forecastday"][day]["day"]["mintemp_c"]
        )
    return {
        "lat": lat,
        "lon": lon,
        "dates": dates,
        "max_temps": max_temps,
        "min_temps": min_temps,
    }

In [55]:
response = parse_forcast_response(get_forecast(lat_lon=(48.5, 2.35)))

In [56]:
pd.DataFrame.from_dict(response)

,lat,lon,dates,max_temps,min_temps
0,48.5,2.35,2023-07-19,28.1,16.2
1,48.5,2.35,2023-07-20,27.1,14.1
2,48.5,2.35,2023-07-21,29.1,14.3


In [62]:
mapping_json = json.loads(
    Path(WEATHER_LOCATION_MAPPING_DIR).read_text(encoding="UTF-8")
)
area_lat_lon = mapping_json["lat_lon"]
grid_zone_mapping = mapping_json["grid_zone"]

In [77]:
ptid_area_mapping = json.loads(Path(PTID_AREA_MAPPING_DIR).read_text(encoding="UTF-8"))

In [78]:
df = pd.DataFrame.from_dict(area_lat_lon).T.rename(columns={0: "Lat", 1: "Lon"})
df["Grid Zone"] = df.index.map(grid_zone_mapping)
df["PTID"] = df["Grid Zone"].map(ptid_area_mapping)
df.head()

,Lat,Lon,Grid Zone,PTID
ALB,42.652580,-73.756233,CAPITL,61757.0
ART,43.974785,-75.910759,MHK VL,61756.0
BGM,42.098843,-75.920647,CENTRL,61754.0
BUF,42.880230,-78.878738,WEST,61752.0
ELM,42.084972,-76.798553,CENTRL,61754.0
